# Day 7: Amplification Circuit

https://adventofcode.com/2019/day/7

## Setup

In [1]:
CODES = parse.(Int64, split(readlines("input/07.txt")[1], ","))
CODES'

1×511 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 3  8  1001  8  10  8  105  1  0  0  …  4  9  3  9  101  2  9  9  4  9  99

## Implementation

In [2]:
# State: anticipate potentially multiple inputs and multiple outputs
mutable struct State
    code::Array{Int}
    inputs::Array{Int}
    outputs::Array{Int}
    pos::Int
    done::Bool
end
State(code::Array{Int}, inputs::Array{Int}) = State(code, inputs, [], 1, false)
State(code::Array{Int}) = State(code, Int[])

# HOLD: do not advance pos
struct Hold end
HOLD = Hold()

# WAIT: need more input to continue
struct Wait end
WAIT = Wait()

# CONTINUE: opposite of wait; run next step immediately
struct Continue end
CONTINUE = Continue()

# arithmetic, jumps, conditionals
ADD(state::State, a, b, out) = state.code[out] = a + b
MUL(state::State, a, b, out) = state.code[out] = a * b
JIF(state::State, a, b) = if a != 0 state.pos = b+1; HOLD else true end 
JIN(state::State, a, b) = if a == 0 state.pos = b+1; HOLD else true end
CLT(state::State, a, b, out) = state.code[out] = 1 * (a < b)
CEQ(state::State, a, b, out) = state.code[out] = 1 * (a == b)

# I/O
INP(state::State, out) = isempty(state.inputs) ? WAIT : state.code[out] = popfirst!(state.inputs)
function OUT(state::State, a)
    push!(state.outputs, a)
    state.pos += 2
    WAIT
end

# all implemented operations
OPFUNCS = Dict(1=>ADD, 2=>MUL, 3=>INP, 4=>OUT, 5=>JIF, 6=>JIN, 7=>CLT, 8=>CEQ)

# deduce number of args by inspection
ARITY(f) = length(first(methods(f)).sig.parameters) - 2

# parse op code and parameter modes
function op_pmode(X)
    X, op = divrem(X, 100)
    X, mode1 = divrem(X, 10)
    X, mode2 = divrem(X, 10)
    X, mode3 = divrem(X, 10)
    return op, [mode1, mode2, mode3]
end

# increment program state
function step(state::State)
    # get func
    code, pos = state.code, state.pos
    op, pmode = op_pmode(state.code[pos])
    # TODO: do we need to special-case halt?
    if op == 99
        state.done = true
        return
    end
    f = OPFUNCS[op]
    # get args accounting for parameter modes
    arity = ARITY(f)
    pmode = pmode[1:arity]
    codes = code[pos+1:pos+arity]
    args = [m==0 ? code[c+1] : c for (m,c) in zip(pmode,codes)]
    # hack: write locs are always (1-indexed) "pointers"
    if f in (ADD, MUL, INP, CLT, CEQ)
        args[end] = codes[end] + 1
    end

    # execute operation, checking whether to increment position
    result = f(state, args...)
    if !( result in (WAIT, HOLD) )
        state.pos = pos + arity + 1
    end
    return result === WAIT ? WAIT : CONTINUE
end

# run program loop until halt or input wait
function runprog(state::State)
    result = CONTINUE
    while !state.done && result != WAIT
        result = step(state)
    end
    return state
end

runprog(code::Array, inputs=Int[]) = runprog(State(copy(code), inputs))
runprog(code::Array, input::Int) = runprog(code, [input])
;

In [3]:
function testthrust(phases::Array{Int}, code)
    # run each amp serially with specified phases
    last_out = 0
    for phase in phases
        last_out = runprog(code, Int[phase, last_out]).outputs[end]
    end
    return last_out
end
;

In [4]:
function testthrust2(phases::Array{Int}, code, maxiter=10000)
    # treat states as a circular queue
    states = [State(copy(code), [phase]) for phase in phases]
    # bootstrap feedback loop
    push!(states[end].outputs, 0)
    prev = states[end]
    # track most recent output
    last_out = 0
    while !isempty(states)
        # run cur with inputs equal to prev's output
        cur = popfirst!(states)
        append!(cur.inputs, prev.outputs)
        prev.outputs = []
        runprog(cur)
        prev = cur
        # track output if *not* done
        # because once everything's done,
        # all output queues will be empty
        if !cur.done
            last_out = cur.outputs[1]
            push!(states, cur)
        end
    end
    return last_out
end
;

## Tests

In [5]:
test1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
test2 = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
test3 = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
;

In [6]:
testthrust([4,3,2,1,0], test1)

43210

In [7]:
testthrust([0,1,2,3,4], test2)

54321

In [8]:
testthrust([1,0,4,3,2], test3)

65210

In [9]:
test4 = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
test5 = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
    -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
    53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
;

In [10]:
testthrust2([9,8,7,6,5], test4)

139629729

In [11]:
testthrust2([9,7,8,5,6], test5)

18216

## Part One

In [12]:
using Combinatorics: permutations

In [13]:
outputs = []
r = 0:4
perms = permutations(r)
@time for perm in perms
    push!(outputs, testthrust(perm, CODES))
end

  0.117694 seconds (341.41 k allocations: 19.393 MiB, 4.83% gc time)


In [14]:
maximum(outputs)

206580

## Part Two

In [15]:
outputs2 = []
r = 5:9
perms = permutations(r)
@time for perm in perms
    push!(outputs2, testthrust2(perm, CODES))
end

  0.129895 seconds (517.04 k allocations: 29.383 MiB, 3.70% gc time)


In [16]:
maximum(outputs2)

2299406